<a href="https://colab.research.google.com/github/dmorton714/data-2-demo/blob/main/week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sqlite3 import connect

## **We are going to read in another data set to answer a new question. *Is there evidence for a correlation between income level and the rate of uninsured for county level data?***

In [2]:
income = pd.read_csv("https://raw.githubusercontent.com/dmorton714/data-2-demo/main/kaggle_income.csv", encoding='latin-1') # this one is encoded in latin-1
income.head()

id  State_Code State_Name State_ab          County            City  \
0  1011000           1    Alabama       AL   Mobile County       Chickasaw   
1  1011010           1    Alabama       AL  Barbour County      Louisville   
2  1011020           1    Alabama       AL   Shelby County      Columbiana   
3  1011030           1    Alabama       AL   Mobile County         Satsuma   
4  1011040           1    Alabama       AL   Mobile County  Dauphin Island   

             Place  Type Primary  Zip_Code Area_Code     ALand     AWater  \
0   Chickasaw city  City   place     36611       251  10894952     909156   
1        Clio city  City   place     36048       334  26070325      23254   
2  Columbiana city  City   place     35051       205  44835274     261034   
3      Creola city  City   place     36572       251  36878729    2374530   
4   Dauphin Island  Town   place     36528       251  16204185  413605152   

         Lat        Lon   Mean  Median  Stdev        sum_w  
0  30.771450 -88.079697  38773   30506  33101  1638.260513  
1  31.708516 -85.611039  37725   19528  43789   258.017685  
2  33.191452 -86.615618  54606   31930  57348   926.031000  
3  30.874343 -88.009442  63919   52814  47707   378.114619  
4  30.250913 -88.171268  77948   67225  54270   282.320328

In [3]:
coverage = pd.read_csv("https://raw.githubusercontent.com/dmorton714/data-2-demo/main/Insurance%20Coverage%20-%20All%20Demographics%20By%20County.csv")
coverage.head()

index      year  statefips state_name  countyfips           County  \
0      0  01-01-08          1    Alabama           0  State ofAlabama   
1      1  01-01-08          1    Alabama           0  State ofAlabama   
2      2  01-01-08          1    Alabama           0  State ofAlabama   
3      3  01-01-08          1    Alabama           1   Autauga County   
4      4  01-01-08          1    Alabama           1   Autauga County   

                    Metric         Value  
0           Number insured  6.540953e+06  
1         Number uninsured  1.181000e+06  
2  Percent Uninsured (All)  2.073000e-01  
3           Number insured  3.847400e+04  
4         Number uninsured  5.880000e+03

Documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

In [4]:
conn = connect(':memory:')
# create tables
# example: dataframe.to_sql("tablename", conn)
income.to_sql("income", conn)
coverage.to_sql("coverage", conn)
# convenient function
def sql(a_string):
    return(pd.read_sql(a_string, conn))

# Coverage Table

1) Find all the 'Metric' types.


In [9]:
# code goes here 
a_string = """select distinct Metric from coverage"""
sql(a_string)

Metric
0           Number insured
1         Number uninsured
2  Percent Uninsured (All)

2) What does 'countyfips' = 0 indicate ?


In [ ]:
a_string = """select * from coverage where countyfips=0"""
sql(a_string)

3) Find the number of counties listed for Alabama. <br>
Question could be more specific... did they want distinct list of counties from income or coverage table?


In [17]:
# code goes here 
a_string = """select count(distinct county) from coverage where state_name = 'Alabama'"""
sql(a_string)

count(distinct county)
0                      68

In [16]:
a_string = """select count(distinct county)  from income where State_Name = 'Alabama'"""
sql(a_string)

count(distinct county)
0                      36

4) Which county in Arkansas has the least number of insured?


In [21]:
# code goes here 
a_string = """select county, min(value) from coverage where Metric = 'Number insured' and state_name = 'Arkansas'"""
sql(a_string)

County  min(value)
0  Calhoun County      3202.0

5) Which state has the the highest number of total uninsured?


In [23]:
# code goes here 
a_string = """select state_name, max(value) from coverage where Metric = 'Number uninsured'"""
sql(a_string)

state_name  max(value)
0  California  12556975.0

# Income Table

6) For the income dataset how many different place types 'Type' are there?


In [28]:
# code goes here 
a_string = """select count(distinct Type) as distinct_Type from income"""
sql(a_string)

distinct_Type
0             10

 7) What is the maximum 'Mean' income for a county in the state of Washington?


In [31]:
# code goes here 
a_string = """select max(mean) from income where State_Name = 'Washington'"""
sql(a_string)

max(mean)
0     187248

8) Which county in the state of Washington has the highest 'Mean' income?


In [32]:
# code goes here 
a_string = """select County, max(mean) from income where State_Name = 'Washington'"""
sql(a_string)

County  max(mean)
0  Adams County     187248

9) How much does the highest 'Mean' income for the state of Washington differ from the highest 'Median' income?


In [35]:
# Let's piecemeal this..We know how to get the max(mean) for state of Washington from above
a_string = """select max(mean) from income where State_Name = 'Washington'"""
sql(a_string)

difference
0      112752

In [34]:
# Do the same for median...
a_string = """select max(median) from income where State_Name = 'Washington'"""
sql(a_string)

max(median)
0       300000

In [36]:
# Now subtract the max values...
a_string = """select (max(median) - max(mean)) as difference
from income where State_Name = 'Washington'"""
sql(a_string)

difference
0      112752

# Join Income and Coverage Tables

10) Perform an inner join of the coverage and income tables on the column 'County'.


In [38]:
# code goes here 
a_string = """select * from income t1
              join coverage t2
              on t1.County = t2.County limit 10"""
sql(a_string)

index       id  State_Code State_Name State_ab         County       City  \
0      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
1      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
2      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
3      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
4      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
5      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
6      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
7      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
8      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   
9      0  1011000           1    Alabama       AL  Mobile County  Chickasaw   

            Place  Type Primary  ...        sum_w level_0  index      year  \
0  Chickasaw city  City   place  ...  1638.260513     147    147  01-01-08   
1  Chickasaw city  City   place  ...  1638.260513     148    148  01-01-08   
2  Chickasaw city  City   place  ...  1638.260513     149    149  01-01-08   
3  Chickasaw city  City   place  ...  1638.260513    9726   9726  01-01-09   
4  Chickasaw city  City   place  ...  1638.260513    9727   9727  01-01-09   
5  Chickasaw city  City   place  ...  1638.260513    9728   9728  01-01-09   
6  Chickasaw city  City   place  ...  1638.260513   19308  19308  01-01-10   
7  Chickasaw city  City   place  ...  1638.260513   19309  19309  01-01-10   
8  Chickasaw city  City   place  ...  1638.260513   19310  19310  01-01-10   
9  Chickasaw city  City   place  ...  1638.260513   28890  28890  01-01-11   

   statefips  state_name  countyfips         County                   Metric  \
0          1     Alabama          97  Mobile County           Number insured   
1          1     Alabama          97  Mobile County         Number uninsured   
2          1     Alabama          97  Mobile County  Percent Uninsured (All)   
3          1     Alabama          97  Mobile County           Number insured   
4          1     Alabama          97  Mobile County         Number uninsured   
5          1     Alabama          97  Mobile County  Percent Uninsured (All)   
6          1     Alabama          97  Mobile County           Number insured   
7          1     Alabama          97  Mobile County         Number uninsured   
8          1     Alabama          97  Mobile County  Percent Uninsured (All)   
9          1     Alabama          97  Mobile County           Number insured   

        Value  
0  290133.000  
1   58969.000  
2       0.169  
3  290991.000  
4   60737.000  
5       0.173  
6  288389.000  
7   66095.000  
8       0.186  
9  287408.000  

[10 rows x 29 columns]

11) Add an order by 'Mean' income to the join statement, and display the first 20 rows.


In [41]:
# to see all columns use this setting
pd.set_option('display.max_columns',None)
a_string = """select * from income t1
              join coverage t2
              on t1.County = t2.County
              order by mean
              limit 20"""
sql(a_string)

index       id  State_Code State_Name State_ab            County     City  \
0      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
1      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
2      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
3      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
4      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
5      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
6      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
7      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
8      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
9      50  1011500           1    Alabama       AL  St. Clair County  Vincent   
10     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
11     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
12     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
13     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
14     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
15     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
16     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
17     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
18     50  1011500           1    Alabama       AL  St. Clair County  Vincent   
19     50  1011500           1    Alabama       AL  St. Clair County  Vincent   

      Place  Type Primary  Zip_Code Area_Code     ALand  AWater        Lat  \
0   Vincent  Town   place     35178       205  53311740  978770  33.383145   
1   Vincent  Town   place     35178       205  53311740  978770  33.383145   
2   Vincent  Town   place     35178       205  53311740  978770  33.383145   
3   Vincent  Town   place     35178       205  53311740  978770  33.383145   
4   Vincent  Town   place     35178       205  53311740  978770  33.383145   
5   Vincent  Town   place     35178       205  53311740  978770  33.383145   
6   Vincent  Town   place     35178       205  53311740  978770  33.383145   
7   Vincent  Town   place     35178       205  53311740  978770  33.383145   
8   Vincent  Town   place     35178       205  53311740  978770  33.383145   
9   Vincent  Town   place     35178       205  53311740  978770  33.383145   
10  Vincent  Town   place     35178       205  53311740  978770  33.383145   
11  Vincent  Town   place     35178       205  53311740  978770  33.383145   
12  Vincent  Town   place     35178       205  53311740  978770  33.383145   
13  Vincent  Town   place     35178       205  53311740  978770  33.383145   
14  Vincent  Town   place     35178       205  53311740  978770  33.383145   
15  Vincent  Town   place     35178       205  53311740  978770  33.383145   
16  Vincent  Town   place     35178       205  53311740  978770  33.383145   
17  Vincent  Town   place     35178       205  53311740  978770  33.383145   
18  Vincent  Town   place     35178       205  53311740  978770  33.383145   
19  Vincent  Town   place     35178       205  53311740  978770  33.383145   

          Lon  Mean  Median  Stdev  sum_w  level_0  index      year  \
0  -86.402544     0       0      0    0.0      174    174  01-01-08   
1  -86.402544     0       0      0    0.0      175    175  01-01-08   
2  -86.402544     0       0      0    0.0      176    176  01-01-08   
3  -86.402544     0       0      0    0.0     2067   2067  01-01-08   
4  -86.402544     0       0      0    0.0     2068   2068  01-01-08   
5  -86.402544     0       0      0    0.0     2069   2069  01-01-08   
6  -86.402544     0       0      0    0.0     3978   3978  01-01-08   
7  -86.402544     0       0      0    0.0     3979   3979  01-01-08   
8  -86.402544     0     

12) Reverse the order of 'Mean' income and display the first 20 rows.


In [42]:
a_string = """select * from income t1
              join coverage t2
              on t1.County = t2.County
              order by mean desc
              limit 20"""
sql(a_string)

index      id  State_Code State_Name State_ab            County  \
0      56  101920           1    Alabama       AL  Jefferson County   
1      56  101920           1    Alabama       AL  Jefferson County   
2      56  101920           1    Alabama       AL  Jefferson County   
3      56  101920           1    Alabama       AL  Jefferson County   
4      56  101920           1    Alabama       AL  Jefferson County   
5      56  101920           1    Alabama       AL  Jefferson County   
6      56  101920           1    Alabama       AL  Jefferson County   
7      56  101920           1    Alabama       AL  Jefferson County   
8      56  101920           1    Alabama       AL  Jefferson County   
9      56  101920           1    Alabama       AL  Jefferson County   
10     56  101920           1    Alabama       AL  Jefferson County   
11     56  101920           1    Alabama       AL  Jefferson County   
12     56  101920           1    Alabama       AL  Jefferson County   
13     56  101920           1    Alabama       AL  Jefferson County   
14     56  101920           1    Alabama       AL  Jefferson County   
15     56  101920           1    Alabama       AL  Jefferson County   
16     56  101920           1    Alabama       AL  Jefferson County   
17     56  101920           1    Alabama       AL  Jefferson County   
18     56  101920           1    Alabama       AL  Jefferson County   
19     56  101920           1    Alabama       AL  Jefferson County   

         City Place  Type Primary  Zip_Code Area_Code     ALand  AWater  \
0   Odenville  Argo  Town   place     35120       205  27893577  150331   
1   Odenville  Argo  Town   place     35120       205  27893577  150331   
2   Odenville  Argo  Town   place     35120       205  27893577  150331   
3   Odenville  Argo  Town   place     35120       205  27893577  150331   
4   Odenville  Argo  Town   place     35120       205  27893577  150331   
5   Odenville  Argo  Town   place     35120       205  27893577  150331   
6   Odenville  Argo  Town   place     35120       205  27893577  150331   
7   Odenville  Argo  Town   place     35120       205  27893577  150331   
8   Odenville  Argo  Town   place     35120       205  27893577  150331   
9   Odenville  Argo  Town   place     35120       205  27893577  150331   
10  Odenville  Argo  Town   place     35120       205  27893577  150331   
11  Odenville  Argo  Town   place     35120       205  27893577  150331   
12  Odenville  Argo  Town   place     35120       205  27893577  150331   
13  Odenville  Argo  Town   place     35120       205  27893577  150331   
14  Odenville  Argo  Town   place     35120       205  27893577  150331   
15  Odenville  Argo  Town   place     35120       205  27893577  150331   
16  Odenville  Argo  Town   place     35120       205  27893577  150331   
17  Odenville  Argo  Town   place     35120       205  27893577  150331   
18  Odenville  Argo  Town   place     35120       205  27893577  150331   
19  Odenville  Argo  Town   place     35120       205  27893577  150331   

          Lat        Lon    Mean  Median  Stdev     sum_w  level_0  index  \
0   33.691576 -86.503766  242857  300000  25317  1.133107      111    111   
1   33.691576 -86.503766  242857  300000  25317  1.133107      112    112   
2   33.691576 -86.503766  242857  300000  25317  1.133107      113    113   
3   33.691576 -86.503766  242857  300000  25317  1.133107      444    444   
4   33.691576 -86.503766  242857  300000  25317  1.133107      445    445   
5   33.691576 -86.503766  242857  300000  25317  1.133107      446    446   
6   33.691576 -86.503766  242857  300000  25317  1.133107      837    837   
7   33.691576 -86.503766  242857  300000  25317  1.133107      838    838   
8   33.691576 -86.503766  242857  300000  25317  1.133107      839    839   
9   33.691576 -86.503766  242857  300000  25317  1.133107     1080   1080   
10  33.691576 -86.503766  242857  300000  25317  1.133107     1081   1081   
11  33.69157

13) Add an order by 'Percent Uninsured (All)', and display the first 20 rows.


In [43]:
a_string = """select * from income t1
              join coverage t2
              on t1.County = t2.County
              where Metric = 'Percent Uninsured (All)'
              order by mean, value
              limit 20"""
sql(a_string)

index        id  State_Code     State_Name State_ab             County  \
0   13407   2501452          25  Massachusetts       MA    Plymouth County   
1   13407   2501452          25  Massachusetts       MA    Plymouth County   
2   12844   2401893          24       Maryland       MD  Washington County   
3   13683  25023430          25  Massachusetts       MA  Barnstable County   
4   13734   2502390          25  Massachusetts       MA  Barnstable County   
5   13738  25023930          25  Massachusetts       MA  Barnstable County   
6   13955  25025890          25  Massachusetts       MA  Barnstable County   
7   13957  25025900          25  Massachusetts       MA  Barnstable County   
8   13407   2501452          25  Massachusetts       MA    Plymouth County   
9   31441  55011447          55      Wisconsin       WI      Monroe County   
10  12844   2401893          24       Maryland       MD  Washington County   
11  13407   2501452          25  Massachusetts       MA    Plymouth County   
12  13683  25023430          25  Massachusetts       MA  Barnstable County   
13  13734   2502390          25  Massachusetts       MA  Barnstable County   
14  13738  25023930          25  Massachusetts       MA  Barnstable County   
15  13955  25025890          25  Massachusetts       MA  Barnstable County   
16  13957  25025900          25  Massachusetts       MA  Barnstable County   
17  12844   2401893          24       Maryland       MD  Washington County   
18  13407   2501452          25  Massachusetts       MA    Plymouth County   
19  12844   2401893          24       Maryland       MD  Washington County   

               City                 Place     Type Primary  Zip_Code  \
0    South Weymouth            Southfield      CDP   place      2190   
1    South Weymouth            Southfield      CDP   place      2190   
2         Boonsboro           Eakles Mill      CDP   place     21713   
3   North Billerica  Barnstable Town city    Track   Track      1862   
4        Gloucester  Barnstable Town city    Track   Track      1930   
5         Cambridge  Barnstable Town city    Track   Track      2139   
6        Roslindale  Barnstable Town city    Track   Track      2131   
7       East Boston  Barnstable Town city    Track   Track      2128   
8    South Weymouth            Southfield      CDP   place      2190   
9           Ontario               Ontario  Village   place     54651   
10        Boonsboro           Eakles Mill      CDP   place     21713   
11   South Weymouth            Southfield      CDP   place      2190   
12  North Billerica  Barnstable Town city    Track   Track      1862   
13       Gloucester  Barnstable Town city    Track   Track      1930   
14        Cambridge  Barnstable Town city    Track   Track      2139   
15       Roslindale  Barnstable Town city    Track   Track      2131   
16      East Boston  Barnstable Town city    Track   Track      2128   
17        Boonsboro           Eakles Mill      CDP   place     21713   
18   South Weymouth            Southfield      CDP   place      2190   
19        Boonsboro           Eakles Mill      CDP   place     21713   

   Area_Code     ALand     AWater        Lat        Lon  Mean  Median  Stdev  \
0        781   2468089       2867  42.150373 -70.929497     0       0      0   
1        781   2468089       2867  42.150373 -70.929497     0       0      0   
2        301    219926          0  39.467067 -77.685960     0       0      0   
3        978  12447312     365418  42.563617 -71.308189     0       0      0   
4        978         0  708060742  42.506748 -70.617130     0       0      0   
5        617    901795     698900  42.357609 -71.091841     0       0      0   
6        617   2713616      19091  42.287994 -71.103404     0       0      0   
7        617    586981     158875  42.388621 -70.993442     0       0      0   
8        781   2468089       2867  42.150373 -70.929497     0       0      0   
9        608   2602844      20077  43.721919 -90.593

14) Reverse the order of  'Percent Uninsured (All)', and display the first 20 rows.


In [57]:
a_string = """select * from income t1
              join coverage t2
              on t1.County = t2.County
              where Metric = 'Percent Uninsured (All)'
              order by mean desc, value desc
              """
dataframe = sql(a_string)
dataframe

index        id  State_Code     State_Name State_ab  \
0           56    101920           1        Alabama       AL   
1          560    201700           2         Alaska       AK   
2          560    201700           2         Alaska       AK   
3           56    101920           1        Alabama       AL   
4          560    201700           2         Alaska       AK   
...        ...       ...         ...            ...      ...   
1115086  13955  25025890          25  Massachusetts       MA   
1115087  13957  25025900          25  Massachusetts       MA   
1115088  13407   2501452          25  Massachusetts       MA   
1115089  12844   2401893          24       Maryland       MD   
1115090  13407   2501452          25  Massachusetts       MA   

                                  County            City  \
0                       Jefferson County       Odenville   
1        Southeast Fairbanks Census Area  Delta Junction   
2        Southeast Fairbanks Census Area  Delta Junction   
3                       Jefferson County       Odenville   
4        Southeast Fairbanks Census Area  Delta Junction   
...                                  ...             ...   
1115086                Barnstable County      Roslindale   
1115087                Barnstable County     East Boston   
1115088                  Plymouth County  South Weymouth   
1115089                Washington County       Boonsboro   
1115090                  Plymouth County  South Weymouth   

                        Place   Type Primary  Zip_Code Area_Code     ALand  \
0                        Argo   Town   place     35120       205  27893577   
1                  Whitestone    CDP   place     99737       907  18298887   
2                  Whitestone    CDP   place     99737       907  18298887   
3                        Argo   Town   place     35120       205  27893577   
4                  Whitestone    CDP   place     99737       907  18298887   
...                       ...    ...     ...       ...       ...       ...   
1115086  Barnstable Town city  Track   Track      2131       617   2713616   
1115087  Barnstable Town city  Track   Track      2128       617    586981   
1115088            Southfield    CDP   place      2190       781   2468089   
1115089           Eakles Mill    CDP   place     21713       301    219926   
1115090            Southfield    CDP   place      2190       781   2468089   

         AWater        Lat         Lon    Mean  Median  Stdev     sum_w  \
0        150331  33.691576  -86.503766  242857  300000  25317  1.133107   
1             0  64.152838 -145.906385  242857  300000  25317  0.866493   
2             0  64.152838 -145.906385  242857  300000  25317  0.866493   
3        150331  33.691576  -86.503766  242857  300000  25317  1.133107   
4             0  64.152838 -145.906385  242857  300000  25317  0.866493   
...         ...        ...         ...     ...     ...    ...       ...   
1115086   19091  42.287994  -71.103404       0       0      0  0.000000   
1115087  158875  42.388621  -70.993442       0       0      0  0.000000   
1115088    2867  42.150373  -70.929497       0       0      0  0.000000   
1115089       0  39.467067  -77.685960       0       0      0  0.000000   
1115090    2867  42.150373  -70.929497       0       0      0  0.000000   

         level_0  index      year  statefips     state_name  countyfips  \
0          16184  16184  01-01-09         40       Oklahoma          67   
1          38603  38603  01-01-12          2         Alaska         240   
2            275    275  01-01-08          2         Alaska         240   
3           6602   6602  01-01-08         40       Oklahoma          67   
4           9857   9857  01-01-09          2         Alaska         240   
...          ...    ...       ...        ...            ...         ...   
1115086    70787  70787  01-01-15         25  Massachusetts           1   
1115087    70787  70787  01-01-15         25  Massachusetts           1   
111508

15) Is there evidence for a correlation between income level and the rate of uninsured for county level data?

In [60]:
# code goes here 
# from matplotlib import pyplot as plt
import seaborn as sns

sample_df = dataframe.sample(n=500)
sns.set_style('dark')
sns.regplot(x='Value',y='Mean', data=sample_df)

<Axes: xlabel='Value', ylabel='Mean'>